In [2]:
# Brightway packages
import bw2io as bi
import bw2data as bd
import pandas as pd
# Brightway type hints
from bw2io import SingleOutputEcospold2Importer
# import/export
import os

In [3]:
# Set up Brightway2 project and necessary filers

from bw2data import projects, databases # for setting project and checking databases
from bw2io import create_default_biosphere3 # for creating biosphere3 if not exists
from filters import CompartmentFilter, CASNumberFilter, NameFilter, apply_filter, normalize_cas
import requests # for downloading files
from io import StringIO # for handling in-memory file objects

projects.set_current("PB_LCIA")

In [4]:
# in this case the database is all biosphere3 flows
# for that reason, we check if it exists and create it if not

if "biosphere3" not in databases:
    create_default_biosphere3()
    print("biosphere3 created.")
else:
    print("biosphere3 already exists.")

print("Databases now:", list(databases))

# Extract biosphere flows into a DataFrame
bio = bd.Database("biosphere3")

rows = []
for flow in bio:
    # Each flow is a dict-like dataset
    rows.append({
        "name": flow.get("name"),
        "categories": flow.get("categories"), # compartment
        "unit": flow.get("unit"),
        "type": flow.get("type"),          # e.g., "emission", "resource", etc.
        "location": flow.get("location"),  # often None for biosphere flows
        "database": flow.get("database"),  # should be "biosphere3"
        "code": flow.get("code"),          # unique identifier inside the database
        "reference_product": flow.get("reference product"),
        "comment": flow.get("comment"),
        "CAS": flow.get("CAS number"),
        "formula": flow.get("formula"),
    })

df = pd.DataFrame(rows)

# Optional: sort for readability
df = df.sort_values(["categories", "name"], na_position="last").reset_index(drop=True)

# 3) Write to Excel
# output_path = "biosphere3_export.xlsx"
# df.to_excel(output_path, index=False)

# print(f"Exported {len(df)} biosphere flows to: {output_path}")

biosphere3 already exists.
Databases now: ['biosphere3']


In [5]:
# First: get necessary data for filtering from external source
# For this the list of GHG defined IPCC in AR6 WGI Chapter 7 is needed
# Missing or unprecise values are updated with values provided by Hodnebrog
# or manually inserted for CO2, CH4, N2O  

In [6]:
# Step 1: Fetch CSV with list of GHG defined by IPCC in AR6 from GitHub
print("\n" + "=" * 60)
print("STEP 1: Fetching list of GHG by IPCC from GitHub")
print("=" * 60)

csv_url = "https://raw.githubusercontent.com/chrisroadmap/ar6/main/data_input/ghg_properties/metrics_supplement.csv"
response = requests.get(csv_url)
response.raise_for_status()

df_ar6 = pd.read_csv(StringIO(response.text))

# Keep only first 6 columns, since the others are irrelevant for our purpose
df_ar6 = df_ar6.iloc[:, :6]

print(f"Total rows in CSV: {len(df_ar6)}")

# Find the correct column names (case-insensitive search) and build rename dictionary
name_col = None
cas_col = None
lifetime_col = None
rad_eff_col = None
formula_col = None
rename_dict = {}

for col in df_ar6.columns:
    col_lower = col.lower()
    if 'name' in col_lower:
        name_col = col
    if 'cas' in col_lower:
        cas_col = col
    if 'lifetime' in col_lower:
        new_name = f"{col} [years]"
        rename_dict[col] = new_name
        lifetime_col = new_name
    if 'radiative' in col_lower or ('rad' in col_lower and 'eff' in col_lower):
        new_name = f"{col} [W m-2 ppb-1]"
        rename_dict[col] = new_name
        rad_eff_col = new_name
    if 'formula' in col_lower:
        formula_col = col

# Apply all renames at once
if rename_dict:
    df_ar6.rename(columns=rename_dict, inplace=True)

mol_col = "Molar_mass [kg mol-1]"

# Add Molar Mass column, the values are substantial for characterization step later
df_ar6[mol_col] = pd.NA


if cas_col is None:
    print("\n ERROR: Could not find CAS column. Available columns:")
    for col in df_ar6.columns:
        print(f"  - {col}")
    raise ValueError("CAS column not found in CSV")

# Normalize CAS numbers in CSV using the function from filters.py 
# this is done to ensure consistent formatting for later matching
df_ar6[cas_col] = df_ar6[cas_col].apply(normalize_cas)

print(df_ar6.to_string())


STEP 1: Fetching list of GHG by IPCC from GitHub
Total rows in CSV: 249
                             # Name         CASRN                Acronym                          Formula Lifetime [years]  Radiative_efficiency [W m-2 ppb-1] Molar_mass [kg mol-1]
0                    Carbon dioxide             0                                                     CO2              nan                               0.000                  <NA>
1                           Methane             0                                                     CH4           11.800                               0.000                  <NA>
2                     Nitrous oxide             0                                                     N2O          109.000                               0.003                  <NA>
3            Trichlorofluoromethane       75-69-4                 CFC-11                            CCl3F           52.000                               0.291                  <NA>
4           Dichlorodi

In [7]:
# Step 2: Add CO2, CH4, N2O entries
print("\n" + "=" * 60)
print("STEP 2: Add Manual GHG CAS Entries")
print("=" * 60)

# Manually add CO2, CH4, N2O data since the radiative efficiency values are imprecise 
# and molar mass values are missing in the AR6 list
ghg_data = {
    "Carbon dioxide": {"cas": "124-38-9", "rad_eff": 1.33e-5, "m_mol":0.04401},
    "Methane": {"cas": "74-82-8", "rad_eff": 0.000388, "m_mol":0.01604},
    "Nitrous oxide": {"cas": "10024-97-2", "rad_eff": 0.0032, "m_mol":0.04401},
}

for substance_name, info in ghg_data.items():
    cas_number = info["cas"]
    rad_val = info["rad_eff"]
    m_mol = info["m_mol"]
    # Iterate through rows in Name column only
    for idx, row in df_ar6.iterrows():
        if name_col and pd.notna(row[name_col]):
            # Exact case-insensitive match
            if str(row[name_col]).strip().lower() == substance_name.lower():
                # Update CAS (keep normalized form)
                df_ar6.loc[idx, cas_col] = normalize_cas(cas_number)
                # Update radiative efficiency: use detected column if present, else create a new column
                df_ar6.loc[idx, rad_eff_col] = rad_val
                # Update molar mass: use detected column if present, else create a new column
                df_ar6.loc[idx, mol_col] = m_mol
                print(f"\n  Updated '{substance_name}' -> CAS {cas_number}, Radiative_Efficiency {rad_val}, Molar_Mass {m_mol}")

print(f"\n {df_ar6.to_string()}")


STEP 2: Add Manual GHG CAS Entries

  Updated 'Carbon dioxide' -> CAS 124-38-9, Radiative_Efficiency 1.33e-05, Molar_Mass 0.04401

  Updated 'Methane' -> CAS 74-82-8, Radiative_Efficiency 0.000388, Molar_Mass 0.01604

  Updated 'Nitrous oxide' -> CAS 10024-97-2, Radiative_Efficiency 0.0032, Molar_Mass 0.04401

                              # Name         CASRN                Acronym                          Formula Lifetime [years]  Radiative_efficiency [W m-2 ppb-1] Molar_mass [kg mol-1]
0                    Carbon dioxide      124-38-9                                                     CO2              nan                            0.000013               0.04401
1                           Methane       74-82-8                                                     CH4           11.800                            0.000388               0.01604
2                     Nitrous oxide    10024-97-2                                                     N2O          109.000                     

In [8]:
# Step 3: Enrich with Hodnebrog et al. (2020) data
print("\n" + "=" * 60)
print("STEP 3: Enrich with Hodnebrog et al. (2020) data")
print("=" * 60)

hodne_url = "https://raw.githubusercontent.com/chrisroadmap/ar6/main/data_input/Hodnebrog_et_al_2020_revgeo/hodnebrog20.csv"
df_hodne = pd.read_csv(hodne_url)

# Find the correct column names (case-insensitive search)
hod_cas_col = None
hod_rad_col = None
hod_mol_col = None
hod_formula_col = None

for col in df_hodne.columns:
    col_lower = col.lower()
    if 'cas' in col_lower:
        hod_cas_col = col
    if 're' in col_lower:
        hod_rad_col = col
    if 'molar' in col_lower:
        hod_mol_col = col
    if 'formula' in col_lower:
        hod_formula_col = col


# Debug: print detected columns
if hod_cas_col is None:
    raise ValueError("CAS column not found in Hodnebrog table")

# Normalize CAS in Hodnebrog table
df_hodne[hod_cas_col] = df_hodne[hod_cas_col].apply(normalize_cas)

# counter for how many entries were updated
updates = 0
mm_updates = 0

for idx, row in df_ar6.iterrows():
    cas_value = row[cas_col]
    matched_row = None
    
    # Try CAS-based matching first (if CAS is valid)
    if pd.notna(cas_value) and cas_value not in ("0", 0):
        cas_matches = df_hodne[df_hodne[hod_cas_col] == cas_value]
        if not cas_matches.empty:
            matched_row = cas_matches.iloc[0]
    
    # Try formula-based matching if CAS is invalid or empty
    if matched_row is None and cas_value in ("0", 0) and formula_col and hod_formula_col and pd.notna(row[formula_col]):
        target_formula = str(row[formula_col]).strip().lower()
        formula_matches = df_hodne[df_hodne[hod_formula_col].astype(str).str.strip().str.lower() == target_formula]
        if not formula_matches.empty:
            matched_row = formula_matches.iloc[0]
    
    # Update values from matched row
    if matched_row is not None:
        # Update radiative efficiency if available and non-zero
        if hod_rad_col and pd.notna(matched_row[hod_rad_col]):
            rad_val = pd.to_numeric(matched_row[hod_rad_col], errors="coerce")
            if pd.notna(rad_val):
                df_ar6.loc[idx, rad_eff_col] = rad_val
                updates += 1
        
        # Update molar mass if available
        if hod_mol_col and pd.notna(matched_row[hod_mol_col]):
            mm_val = pd.to_numeric(matched_row[hod_mol_col], errors="coerce")
            if pd.notna(mm_val):
                df_ar6.loc[idx, mol_col] = mm_val
                mm_updates += 1

print(f"Updated radiative efficiency from Hodnebrog for {updates} rows.")
print(f"Updated molar mass from Hodnebrog for {mm_updates} rows.")

# List entities with no molar mass
df_no_molar_mass = df_ar6[df_ar6[mol_col].isna()]
print(f"\nRemaining entities with no molar mass: {len(df_no_molar_mass)}")

print(df_ar6.to_string())


STEP 3: Enrich with Hodnebrog et al. (2020) data
Updated radiative efficiency from Hodnebrog for 246 rows.
Updated molar mass from Hodnebrog for 246 rows.

Remaining entities with no molar mass: 0
                             # Name         CASRN                Acronym                          Formula Lifetime [years]  Radiative_efficiency [W m-2 ppb-1] Molar_mass [kg mol-1]
0                    Carbon dioxide      124-38-9                                                     CO2              nan                            0.000013               0.04401
1                           Methane       74-82-8                                                     CH4           11.800                            0.000388               0.01604
2                     Nitrous oxide    10024-97-2                                                     N2O          109.000                            0.003200               0.04401
3            Trichlorofluoromethane       75-69-4                 CFC-11      

In [9]:
# Now preparing the necessary data for filtering is done
# Second step is to apply the filters to the database for characterization

In [10]:
'continue here'

'continue here'

In [11]:
# Step 1: Filter your database for air compartment

print("=" * 60)
print("STEP 1: Filter for Air Compartment")
print("=" * 60)

df_air = apply_filter(df, CompartmentFilter("air"))
print(f"Total flows in 'air' compartment: {len(df_air)}")

STEP 1: Filter for Air Compartment
Total flows in 'air' compartment: 1877


In [12]:
# Step 2: Split air flows by CAS presence
print("\n" + "=" * 60)
print("STEP 2: Split Air Flows by CAS Presence")
print("=" * 60)

# Find the correct column names (case-insensitive search)
flow_cas_col = None

for col in df.columns:
    col_lower = col.lower()
    if 'cas' in col_lower:
        flow_cas_col = col

df_air_with_cas = apply_filter(df_air, CASNumberFilter(cas=True))
# Normalize CAS numbers in df_air_with_cas
df_air_with_cas[flow_cas_col] = df_air_with_cas[flow_cas_col].apply(normalize_cas)

df_air_without_cas = apply_filter(df_air, CASNumberFilter(cas=False))

print(f"Air flows WITH CAS number: {len(df_air_with_cas)}")
print(f"Air flows WITHOUT CAS number: {len(df_air_without_cas)}")




STEP 2: Split Air Flows by CAS Presence
Air flows WITH CAS number: 1742
Air flows WITHOUT CAS number: 135


In [13]:
# Step 3: Match air flows with CAS to CSV data
print("\n" + "=" * 60)
print("STEP 3: Matching Air Flows (with CAS) to CSV")
print("=" * 60)

matching_flows = []
non_matching_flows = []

for idx, row in df_air_with_cas.iterrows():
    cas_flow = row[flow_cas_col]
    
    # Find if this CAS matches any in the CSV
    csv_match = df_ar6[df_ar6[cas_col] == cas_flow]
    
    if not csv_match.empty:
        # Found a match
        match_row = csv_match.iloc[0]
        flow_data = row.to_dict()
        flow_data['Lifetime [years]'] = pd.to_numeric(match_row.get(lifetime_col), errors='coerce') if lifetime_col else None
        flow_data['Radiative_Efficiency [W m-2 ppb-1]'] = pd.to_numeric(match_row.get(rad_eff_col), errors='coerce') if rad_eff_col else None
        flow_data['Molar_Mass [kg mol-1]'] = pd.to_numeric(match_row.get(mol_col), errors='coerce') if mol_col else None
        matching_flows.append(flow_data)
    else:
        # No match found
        non_matching_flows.append(row)

# Create output dataframes
df_air_matched = pd.DataFrame(matching_flows) if matching_flows else pd.DataFrame()
df_air_not_matched = pd.DataFrame(non_matching_flows) if non_matching_flows else pd.DataFrame()

print(f"\nAir flows with CAS that MATCH CSV: {len(df_air_matched)}")
print(f"Air flows with CAS that DON'T MATCH CSV: {len(df_air_not_matched)}")

if len(df_air_matched) > 0:
    print("\nMatching air flows (first 5):")
    print(df_air_matched.to_string())
else:
    print("\nNo matching air flows")

if len(df_air_not_matched) > 0:
    print("\nNon-matching air flows with CAS (first 5):")
    print(df_air_not_matched[['name', 'CAS']].head())
else:
    print("\nAll air flows with CAS matched!")

print(f"\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"Total air flows: {len(df_air)}")
print(f"  ├─ With CAS: {len(df_air_with_cas)}")
print(f"  │  ├─ Matched with CSV: {len(df_air_matched)}")
print(f"  │  └─ Not matched: {len(df_air_not_matched)}")
print(f"  └─ Without CAS: {len(df_air_without_cas)}")


STEP 3: Matching Air Flows (with CAS) to CSV

Air flows with CAS that MATCH CSV: 229
Air flows with CAS that DON'T MATCH CSV: 1513

Matching air flows (first 5):
                                                   name                                     categories      unit              type location    database                                  code reference_product comment         CAS formula  Lifetime [years]  Radiative_Efficiency [W m-2 ppb-1]  Molar_Mass [kg mol-1]
0                                          Bromopropane                                         (air,)  kilogram          emission     None  biosphere3  f2e4394f-61bb-5493-983d-d3d5b7b96a41              None    None    106-94-5    None             0.041                            0.002340                0.12300
1                                                Butane                                         (air,)  kilogram          emission     None  biosphere3  982b0510-96ac-4bcb-a758-e98006b95f4d              None    

In [ ]:
import requests
import time
import random
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Filter unmatched air flows: organic compounds with boiling point < 250°C
print("\n" + "=" * 60)
print("STEP 4: Filter Unmatched Air Flows by PubChem Properties")
print("=" * 60)

# Gentle request session with pooling, retries, and a friendly User-Agent
session = requests.Session()
session.headers.update({
    "User-Agent": "PB-LCIA/1.0 (+https://example.org)",
    "Accept": "application/json"
})
retry_strategy = Retry(
    total=3,
    status_forcelist=[429, 502, 503, 504],
    allowed_methods=["GET"],
    backoff_factor=0.5,
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

# Simple in-memory caches to avoid repeated calls within a run
synonym_cache = {}
compound_cache = {}

# Rate limiter to avoid hitting PubChem too fast
class RateLimiter:
    def __init__(self, min_interval=1.2):
        self.min_interval = min_interval
        self._last = 0.0
        self._lock = threading.Lock()
    def wait(self):
        with self._lock:
            now = time.time()
            elapsed = now - self._last
            if elapsed < self.min_interval:
                time.sleep(self.min_interval - elapsed)
            self._last = time.time()

rate_limiter = RateLimiter(min_interval=1.2)

# Retry helper with exponential backoff for transient errors (503, 502, 429) and Retry-After
def get_json_with_retry(url, session, rate_limiter, max_attempts=7, base_delay=1.2):
    last_err = "Unknown error"
    for attempt in range(1, max_attempts + 1):
        try:
            rate_limiter.wait()
            resp = session.get(url, timeout=15)
            # Success
            if resp.status_code == 200:
                return resp.json()
            # Respect Retry-After when present (429/503/502)
            if resp.status_code in (503, 502, 429):
                last_err = f"HTTP {resp.status_code}"
                retry_after = resp.headers.get("Retry-After")
                if retry_after:
                    try:
                        delay = float(retry_after)
                        time.sleep(delay)
                    except ValueError:
                        pass
                jitter = random.uniform(0, 0.4)
                delay = min(base_delay * (2 ** (attempt - 1)) + jitter, 12.0)
                time.sleep(delay)
                continue
            # Other errors: stop early
            return {"__error__": f"API error: {resp.status_code}"}
        except requests.exceptions.RequestException as e:
            # Network-level transient failure
            last_err = str(e)
            jitter = random.uniform(0, 0.4)
            delay = min(base_delay * (2 ** (attempt - 1)) + jitter, 12.0)
            time.sleep(delay)
    # Exhausted attempts
    return {"__error__": f"Request failed after retries: {last_err}"}


def check_pubchem_properties(cas_number, name, session):
    """Check if compound is organic with boiling point < 250°C.
    - Ensures CAS string type
    - Finds CID via name
    - Validates CAS appears in synonyms
    - Reads IsOrganic + Boiling Point from compound props
    """
    cas_number = str(cas_number).strip()
    name_key = str(name).strip()

    # Use cached compound if available
    compound = compound_cache.get(name_key)
    cid = None

    if compound is None:
        # Search by compound name first (more tolerant than CAS)
        search_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{requests.utils.quote(name_key)}/JSON"
        data = get_json_with_retry(search_url, session, rate_limiter)
        if isinstance(data, dict) and "__error__" in data:
            return {"error": data["__error__"], "is_organic": False, "boiling_point": None, "formula": None}
        compound = (data.get('PC_Compounds') or [{}])[0]
        compound_cache[name_key] = compound

    # Extract CID if present
    cid = compound.get('id', {}).get('id', {}).get('cid')

    # Validate CAS via synonyms
    cas_found = False
    if cid:
        if cid in synonym_cache:
            synonyms = synonym_cache[cid]
        else:
            syn_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/synonyms/JSON"
            syn_data = get_json_with_retry(syn_url, session, rate_limiter)
            if isinstance(syn_data, dict) and "__error__" in syn_data:
                return {"error": syn_data["__error__"], "is_organic": False, "boiling_point": None, "formula": None}
            synonyms = (syn_data.get('InformationList', {}).get('Information') or [{}])[0].get('Synonym', [])
            synonym_cache[cid] = synonyms
        cas_found = cas_number in [str(s).strip() for s in synonyms]

    if not cas_found:
        return {"error": f"CAS {cas_number} not found in synonyms", "is_organic": False, "boiling_point": None, "formula": None}

    # Get organic flag and boiling point
    is_organic = compound.get('IsOrganic', False)
    boiling_point = None
    formula = None

    if 'props' in compound:
        for prop in compound['props']:
            urn_label = prop.get('urn', {}).get('label', '').lower()
            val = prop.get('value', {})
            if 'molecular formula' in urn_label:
                formula = val.get('sval', '')
            if 'boiling point' in urn_label:
                bp_value = val.get('fval')
                if bp_value:
                    boiling_point = bp_value

    return {
        'is_organic': is_organic,
        'boiling_point': boiling_point,
        'formula': formula,
        'error': None
    }

# Prepare data for parallel processing - ensure CAS is string
df_air_not_matched_copy = df_air_not_matched.copy()
df_air_not_matched_copy['CAS'] = df_air_not_matched_copy['CAS'].astype(str).str.strip()

rows_to_check = [(idx, row) for idx, row in df_air_not_matched_copy.iterrows()]
filtered_elements = []

# Use ThreadPoolExecutor for parallel requests with gentler concurrency
print(f"\nQuerying PubChem API for {len(rows_to_check)} compounds (parallel with retries + rate limiting)...")

with ThreadPoolExecutor(max_workers=2) as executor:
    future_to_idx = {
        executor.submit(check_pubchem_properties, row['CAS'], row['name'], session): idx
        for idx, row in rows_to_check
    }
    processed = 0
    for future in as_completed(future_to_idx):
        idx = future_to_idx[future]
        row = df_air_not_matched_copy.loc[idx]
        cas_number = row['CAS']
        name = row['name']
        try:
            result = future.result()
            is_organic = result['is_organic']
            boiling_point = result['boiling_point']
            formula = result['formula']
            error = result['error']
            if error:
                print(f"✗ {name} (CAS: {cas_number}) - {error}")
            elif is_organic and (boiling_point is None or boiling_point < 250):
                filtered_row = row.to_dict()
                filtered_row['boiling_point_C'] = boiling_point
                filtered_row['molecular_formula'] = formula
                filtered_elements.append(filtered_row)
                print(f"✓ {name} (CAS: {cas_number}) - Organic, BP: {boiling_point}°C")
            else:
                status = "Not organic" if not is_organic else f"BP: {boiling_point}°C (≥250°C)"
                print(f"✗ {name} (CAS: {cas_number}) - {status}")
        except Exception as e:
            print(f"✗ Processing error for {name}: {str(e)}")
        # Small periodic pause to stay friendly to the API
        processed += 1
        if processed % 4 == 0:
            time.sleep(1.0)

# Create result dataframe
df_air_organic_volatile = pd.DataFrame(filtered_elements) if filtered_elements else pd.DataFrame()

print(f"\n" + "=" * 60)
print(f"Results: {len(df_air_organic_volatile)} compounds match criteria")
print("=" * 60)

if len(df_air_organic_volatile) > 0:
    print("\nOrganic compounds with boiling point < 250°C:")
    print(df_air_organic_volatile[['name', 'CAS', 'boiling_point_C', 'molecular_formula']].to_string())
else:
    print("\nNo compounds match the criteria.")


STEP 4: Filter Unmatched Air Flows by PubChem Properties



Querying PubChem API for 1513 compounds (parallel with retries)...
✗ 1,2-Dichlorobenzene (CAS: 95-50-1) - Request failed after retries: HTTP 503
✗ 1,4-Butanediol (CAS: 110-63-4) - Request failed after retries: HTTP 503
✗ 1-Pentanol (CAS: 71-41-0) - Request failed after retries: HTTP 503
✗ 1-Pentene (CAS: 109-67-1) - Request failed after retries: HTTP 503
✗ 2,2,4-Trimethyl pentane (CAS: 540-84-1) - Request failed after retries: HTTP 503
✗ 2,4-D dimethylamine salt (CAS: 2008-39-1) - Request failed after retries: HTTP 503
✗ 2-Aminopropanol (CAS: 2749-11-3) - Request failed after retries: HTTP 503
✗ 2-Methyl-1-propanol (CAS: 78-83-1) - Request failed after retries: HTTP 503
✗ 2-Methyl-2-butene (CAS: 513-35-9) - Request failed after retries: HTTP 503
✗ 2-Methylpentane (CAS: 107-83-5) - Request failed after retries: HTTP 503
✗ 2-Nitrobenzoic acid (CAS: 552-16-9) - Request failed after retries: HTTP 503
✗ 2-Propanol (CAS: 67-63-0) - Request failed after retries: HTTP 503
✗ 3-Methyl-1-butanol

In [20]:
print(df_air_not_matched.to_string())

                                                          name                                     categories            unit              type location    database                                  code reference_product comment          CAS formula
0                                          1,2-Dichlorobenzene                                         (air,)        kilogram          emission     None  biosphere3  b1c36287-329c-49f0-93c2-68246d34007c              None    None      95-50-1    None
1                                               1,4-Butanediol                                         (air,)        kilogram          emission     None  biosphere3  09db39be-d9a6-4fc3-8d25-1f80b23e9131              None    None     110-63-4    None
2                                                   1-Pentanol                                         (air,)        kilogram          emission     None  biosphere3  048baf1e-6cdc-44a5-92e2-32d15ff54885              None    None      71-41-0    None


In [ ]:
print(f"CSV loaded with columns: {df_ar6.columns.tolist()}")
print(f"Total rows in CSV: {len(df_ar6)}")
print(df_ar6.head())

EXAMPLE 1: Single Filters

Flows in 'air' compartment: 1877
                      name categories          CAS
0      1,2-Dichlorobenzene     (air,)  000095-50-1
1           1,4-Butanediol     (air,)  000110-63-4
2               1-Pentanol     (air,)  000071-41-0
3                1-Pentene     (air,)  000109-67-1
4  2,2,4-Trimethyl pentane     (air,)     540-84-1

Flows with CAS number: 4120
                      name categories          CAS
0      1,2-Dichlorobenzene     (air,)  000095-50-1
1           1,4-Butanediol     (air,)  000110-63-4
2               1-Pentanol     (air,)  000071-41-0
3                1-Pentene     (air,)  000109-67-1
4  2,2,4-Trimethyl pentane     (air,)     540-84-1

Flows without CAS number: 589


In [ ]:
print(df_with_cas[["name", "categories", "CAS"]].head())

                      name categories          CAS
0      1,2-Dichlorobenzene     (air,)  000095-50-1
1           1,4-Butanediol     (air,)  000110-63-4
2               1-Pentanol     (air,)  000071-41-0
3                1-Pentene     (air,)  000109-67-1
4  2,2,4-Trimethyl pentane     (air,)     540-84-1


In [ ]:
# Example: Apply filters sequentially
print("=" * 60)
print("EXAMPLE 2: Sequential Filter Application")
print("=" * 60)


# Start with all flows
df_filtered = df.copy()
print(f"\nStart with: {len(df_filtered)} flows")

# Apply first filter: air compartment
df_filtered = apply_filter(df_filtered, CompartmentFilter("air"))
print(f"After filtering for 'air' compartment: {len(df_filtered)} flows")

# Apply second filter: has CAS number
df_filtered = apply_filter(df_filtered, CASNumberFilter(cas=True))
print(f"After filtering for CAS number: {len(df_filtered)} flows")

print("\nFinal results (air compartment AND has CAS):")
print(df_filtered[["name", "categories", "CAS"]].head(10))


EXAMPLE 2: Sequential Filter Application

Start with: 4709 flows
After filtering for 'air' compartment: 1877 flows
After filtering for CAS number: 1742 flows

Final results (air compartment AND has CAS):
                       name categories          CAS
0       1,2-Dichlorobenzene     (air,)  000095-50-1
1            1,4-Butanediol     (air,)  000110-63-4
2                1-Pentanol     (air,)  000071-41-0
3                 1-Pentene     (air,)  000109-67-1
4   2,2,4-Trimethyl pentane     (air,)     540-84-1
5  2,4-D dimethylamine salt     (air,)  002008-39-1
6           2-Aminopropanol     (air,)  002749-11-3
7       2-Methyl-1-propanol     (air,)  000078-83-1
8         2-Methyl-2-butene     (air,)  000513-35-9
9           2-Methylpentane     (air,)  000107-83-5


In [ ]:
# Example: Different sequential combinations
print("=" * 60)
print("EXAMPLE 3: Different Filter Combinations")
print("=" * 60)

# Combination 1: Water compartment WITHOUT CAS number
df_combo1 = df.copy()
df_combo1 = apply_filter(df_combo1, CompartmentFilter("water"))
df_combo1 = apply_filter(df_combo1, CASNumberFilter(cas=False))
print(f"\nWater compartment WITHOUT CAS number: {len(df_combo1)} flows")
print(df_combo1[["name", "categories", "CAS"]].head(5))

# Combination 2: Flows with 'carbon' in name AND with CAS number
df_combo2 = df.copy()
df_combo2 = apply_filter(df_combo2, NameFilter("carbon"))
df_combo2 = apply_filter(df_combo2, CASNumberFilter(cas=True))
print(f"\n'Carbon' in name AND has CAS number: {len(df_combo2)} flows")
print(df_combo2[["name", "CAS"]].head(10))

# Combination 3: Soil compartment
df_combo3 = df.copy()
df_combo3 = apply_filter(df_combo3, CompartmentFilter("soil"))
print(f"\nSoil compartment: {len(df_combo3)} flows")
print(df_combo3[["name", "categories"]].head(10))


EXAMPLE 3: Different Filter Combinations

Water compartment WITHOUT CAS number: 174 flows
                                                name  \
0  Energy, potential (in hydropower reservoir), c...   
1                         Volume occupied, reservoir   
2                    AOX, Adsorbable Organic Halogen   
3                Actinides, radioactive, unspecified   
4                                       Antimony ion   

                     categories   CAS  
0  (natural resource, in water)  None  
1  (natural resource, in water)  None  
2                      (water,)  None  
3                      (water,)  None  
4                      (water,)  None  

'Carbon' in name AND has CAS number: 95 flows
                                          name          CAS
0                           Ammonium carbonate  000506-87-6
1                       Carbon dioxide, fossil  000124-38-9
2   Carbon dioxide, from soil or biomass stock  000124-38-9
3                   Carbon dioxide, non-fossil

In [ ]:
df_combo2_sorted = df_combo2.sort_values("CAS")
print(df_combo2_sorted[["name", "CAS"]].to_string())

                                               name          CAS
71                                      Bicarbonate  000071-52-3
17                                 Carbon disulfide  000075-15-0
50                                 Carbon disulfide  000075-15-0
72                                 Carbon disulfide  000075-15-0
77                                 Carbon disulfide  000075-15-0
82                                 Carbon disulfide  000075-15-0
28                                 Carbon disulfide  000075-15-0
91                                 Carbon disulfide  000075-15-0
4                                  Carbon disulfide  000075-15-0
39                                 Carbon disulfide  000075-15-0
87                                 Carbon disulfide  000075-15-0
58                           Carbon dioxide, in air  000124-38-9
38                       Carbon dioxide, non-fossil  000124-38-9
27                       Carbon dioxide, non-fossil  000124-38-9
26       Carbon dioxide, 